# Analyze NASDAQ ITCH protocol

In [1]:
import gzip
import shutil
from pathlib import Path
from urllib.request import urlretrieve
from urllib.parse import urljoin
from datetime import datetime
from struct import unpack
from collections import namedtuple, Counter
from datetime import timedelta
from time import time

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

## Download Sample Data

In [ ]:
URL = 'https://emi.nasdaq.com/ITCH/Nasdaq%20ITCH/'
SOURCE_FILE = '03272019.NASDAQ_ITCH50.gz' # .gz 5.13GB, unzipped 11.99GB

data_path = Path('data')
itch_store = str(data_path / 'itch.h5')
order_book_store = data_path / 'order_book.h5'

url = urljoin(URL, SOURCE_FILE)
filename = data_path / url.split('/')[-1]
if not data_path.exists():
    print('Creating directory')
    data_path.mkdir()
if not filename.exists():
    print('Downloading...', url)
    urlretrieve(url, filename)
unzipped = data_path / (filename.stem + '.bin')
if not (data_path / unzipped).exists():
    print('Unzipping to', unzipped)
    with gzip.open(str(filename), 'rb') as f_in:
        with open(unzipped, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)


Creating directory
Downloading... https://emi.nasdaq.com/ITCH/Nasdaq%20ITCH/03272019.NASDAQ_ITCH50.gz
Unzipping to data/03272019.NASDAQ_ITCH50.bin


## Message Formats

In [ ]:
# https://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/NQTVITCHSpecification.pdf

In [2]:
event_codes = {'O': 'Start of Messages',
               'S': 'Start of System Hours',
               'Q': 'Start of Market Hours',
               'M': 'End of Market Hours',
               'E': 'End of System Hours',
               'C': 'End of Messages'}

encoding = {'primary_market_maker': {'Y': 1, 'N': 0},
            'printable'           : {'Y': 1, 'N': 0},
            'buy_sell_indicator'  : {'B': 1, 'S': -1},
            'cross_type'          : {'O': 0, 'C': 1, 'H': 2},
            'imbalance_direction' : {'B': 0, 'S': 1, 'N': 0, 'O': -1}}

formats = {
    ('integer', 2): 'H',
    ('integer', 4): 'I',
    ('integer', 6): '6s',
    ('integer', 8): 'Q',
    ('alpha', 1)  : 's',
    ('alpha', 2)  : '2s',
    ('alpha', 4)  : '4s',
    ('alpha', 8)  : '8s',
    ('price_4', 4): 'I',
    ('price_8', 8): 'Q',
}


In [3]:
message_types_url = 'https://github.com/PacktPublishing/Hands-On-Machine-Learning-for-Algorithmic-Trading/raw/refs/heads/master/Chapter02/01_NASDAQ_TotalView-ITCH_Order_Book/message_types.xlsx'

message_types = (pd.read_excel(message_types_url, sheet_name='messages')
                .sort_values('id').drop('id', axis=1))

message_types.head()

,Name,Offset,Length,Value,Notes
0,Message Type,0,1,S,System Event Message
1,Stock Locate,1,2,Integer,Always 0
2,Tracking Number,3,2,Integer,Nasdaq internal tracking number
3,Timestamp,5,6,Integer,Nanoseconds since midnight
4,Event Code,11,1,Alpha,See System Event Codes below


In [ ]:
message_types.columns

Index(['Name ', 'Offset ', 'Length ', 'Value ', 'Notes '], dtype='object')

In [4]:
cleaning = lambda row: row.strip().lower()\
                          .replace(' ', '_')\
                          .replace('-', '_')\
                          .replace('/', '_')

def clean_message_types(df):
    df.columns = [c.lower().strip() for c in df.columns]
    df['value'] = df['value'].str.strip()
    df['name'] = df['name'].map(cleaning)
    df['notes'] = df['notes'].str.strip()
    df['message_type'] = df.loc[df['name'] == 'message_type', 'value']
    return df

In [5]:
message_types = clean_message_types(message_types)
message_types.head()

,name,offset,length,value,notes,message_type
0,message_type,0,1,S,System Event Message,S
1,stock_locate,1,2,Integer,Always 0,NaN
2,tracking_number,3,2,Integer,Nasdaq internal tracking number,NaN
3,timestamp,5,6,Integer,Nanoseconds since midnight,NaN
4,event_code,11,1,Alpha,See System Event Codes below,NaN


In [ ]:
message_types.groupby('message_type').count()

,name,offset,length,value,notes
message_type,,,,,
A,1,1,1,1,1
B,1,1,1,1,1
C,1,1,1,1,1
D,1,1,1,1,1
E,1,1,1,1,1
F,1,1,1,1,1
H,1,1,1,1,1
I,1,1,1,1,1
J,1,1,1,1,1


In [6]:
cleaning = lambda row: row.lower()\
                          .replace('message', '')\
                          .replace('.', '')\
                          .strip().replace(' ', '_')


message_labels = (message_types.loc[:, ['message_type', 'notes']]
                  .dropna()
                  .rename(columns={'notes': 'name'}))
message_labels['name'] = message_labels['name'].map(cleaning)

message_labels.head()

,message_type,name
0,S,system_event
5,R,stock_directory
23,H,stock_trading_action
31,Y,reg_sho_short_sale_price_test_restricted_indic...
37,L,market_participant_position


In [7]:
cleaning = lambda row: row.lower()\
                          .replace(' ', '_')\
                          .replace('(', '')\
                          .replace(')', '')

message_types['message_type'] = message_types['message_type'].ffill()
message_types = message_types[message_types['name'] != 'message_type']
message_types['value'] = message_types['value'].map(cleaning)

message_types.head()

<ipython-input-7-4634f44d8f36>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  message_types['value'] = message_types['value'].map(cleaning)


,name,offset,length,value,notes,message_type
1,stock_locate,1,2,integer,Always 0,S
2,tracking_number,3,2,integer,Nasdaq internal tracking number,S
3,timestamp,5,6,integer,Nanoseconds since midnight,S
4,event_code,11,1,alpha,See System Event Codes below,S
6,stock_locate,1,2,integer,Locate Code uniquely assigned to the security ...,R


In [ ]:
message_types.info()

<class 'pandas.core.frame.DataFrame'>
Index: 152 entries, 1 to 172
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          152 non-null    object
 1   offset        152 non-null    int64 
 2   length        152 non-null    int64 
 3   value         152 non-null    object
 4   notes         152 non-null    object
 5   message_type  152 non-null    object
dtypes: int64(2), object(4)
memory usage: 8.3+ KB


In [8]:
message_types.loc[:, 'formats'] = (message_types[['value', 'length']]
                                   .apply(tuple, axis=1).map(formats))

In [11]:
message_types.head()

,name,offset,length,value,notes,message_type,formats
1,stock_locate,1,2,integer,Always 0,S,H
2,tracking_number,3,2,integer,Nasdaq internal tracking number,S,H
3,timestamp,5,6,integer,Nanoseconds since midnight,S,6s
4,event_code,11,1,alpha,See System Event Codes below,S,s
6,stock_locate,1,2,integer,Locate Code uniquely assigned to the security ...,R,H


In [17]:
alpha_fields = message_types[message_types['value'] == 'alpha'].set_index('name')
alpha_msgs = alpha_fields.groupby('message_type')
alpha_formats = {k: v.to_dict() for k, v in alpha_msgs['formats']}
alpha_length = {k: v.add(5).to_dict() for k, v in alpha_msgs['length']}

In [19]:
alpha_length

{'A': {'buy_sell_indicator': 6, 'stock': 13},
 'C': {'printable': 6},
 'F': {'buy_sell_indicator': 6, 'stock': 13, 'attribution': 9},
 'H': {'stock': 13, 'trading_state': 6, 'reserved': 6, 'reason': 9},
 'I': {'imbalance_direction': 6,
  'stock': 13,
  'cross_type': 6,
  'price_variation_indicator': 6},
 'J': {'stock': 13},
 'K': {'stock': 13, 'ipo_quotation_release_qualifier': 6},
 'L': {'mpid': 9,
  'stock': 13,
  'primary_market_maker': 6,
  'market_maker_mode': 6,
  'market_participant_state': 6},
 'P': {'buy_sell_indicator': 6, 'stock': 13},
 'Q': {'stock': 13, 'cross_type': 6},
 'R': {'stock': 13,
  'market_category': 6,
  'financial_status_indicator': 6,
  'round_lots_only': 6,
  'issue_classification': 6,
  'issue_sub_type': 7,
  'authenticity': 6,
  'short_sale_threshold_indicator': 6,
  'ipo_flag': 6,
  'luld_reference_price_tier': 6,
  'etp_flag': 6,
  'inverse_indicator': 6},
 'S': {'event_code': 6},
 'W': {'breached_level': 6},
 'Y': {'stock': 13, 'reg_sho_action': 6},
 'h

In [20]:
alpha_formats

{'A': {'buy_sell_indicator': 's', 'stock': '8s'},
 'C': {'printable': 's'},
 'F': {'buy_sell_indicator': 's', 'stock': '8s', 'attribution': '4s'},
 'H': {'stock': '8s', 'trading_state': 's', 'reserved': 's', 'reason': '4s'},
 'I': {'imbalance_direction': 's',
  'stock': '8s',
  'cross_type': 's',
  'price_variation_indicator': 's'},
 'J': {'stock': '8s'},
 'K': {'stock': '8s', 'ipo_quotation_release_qualifier': 's'},
 'L': {'mpid': '4s',
  'stock': '8s',
  'primary_market_maker': 's',
  'market_maker_mode': 's',
  'market_participant_state': 's'},
 'P': {'buy_sell_indicator': 's', 'stock': '8s'},
 'Q': {'stock': '8s', 'cross_type': 's'},
 'R': {'stock': '8s',
  'market_category': 's',
  'financial_status_indicator': 's',
  'round_lots_only': 's',
  'issue_classification': 's',
  'issue_sub_type': '2s',
  'authenticity': 's',
  'short_sale_threshold_indicator': 's',
  'ipo_flag': 's',
  'luld_reference_price_tier': 's',
  'etp_flag': 's',
  'inverse_indicator': 's'},
 'S': {'event_code'

In [21]:
message_fields, fstring = {}, {}
for t, message in message_types.groupby('message_type'):
    message_fields[t] = namedtuple(typename=t, field_names=message.name.tolist())
    fstring[t] = '>' + ''.join(message.formats.tolist())


In [24]:
def format_alpha(mtype, data):
    """Process byte strings of type alpha"""

    for col in alpha_formats.get(mtype).keys():
        if mtype != 'R' and col == 'stock':
            data = data.drop(col, axis=1)
            continue
        data.loc[:, col] = data.loc[:, col].str.decode("utf-8").str.strip()
        if encoding.get(col):
            data.loc[:, col] = data.loc[:, col].map(encoding.get(col))
    return data
